**Install the ChromaDB and the Gemini API Python library**

In [ ]:
!pip install -U -q google.generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 44

**Import the model.**

In [7]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

**Grab an API Key**

將密鑰API處存在左側鑰匙符號底下 （超酷的第一次用這個功能！！）

In [9]:
# Or use `os.getenv('API_KEY')` to fetch an environment variable.
API_KEY=userdata.get('API_KEY')

genai.configure(api_key=API_KEY)

In [10]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [11]:
#將BBC報導關於台灣0403大地震的新聞放進裡面作為素材
DOCUMENT1 = "You can see it here in the city of Hualien. Camera teams are crowded around a 10-storey building leaning over at a terrifying angle. It all looks bizarre and scary. But that was only one of a handful of buildings which suffered structurally, in a city of tens of thousands.A hundred metres away beyond the police cordon, the streets of Hualien look entirely normal. Shops and cafés are open, traffic is flowing. Drive through the city and if you didn't know a big quake had struck days ago, you wouldn't guess it.The fact that this city has survived largely unscathed has sparked immediate discussion of how and why."
DOCUMENT2 = "Just over a year ago, we saw earthquakes of about the same magnitudes striking Turkey and Syria, causing the deaths of more than 50,000 people. These countries, of course, had far fewer resources. But when a much smaller 6.7 magnitude quake hit the city of Christchurch in New Zealand in 2011, almost the entire city centre was flattened.Taiwan is also often on the fault lines, but it has made significant progress in coping with shakes. Many say the wake-up call was Taiwan's 1999 Chi Chi earthquake - the worst in its history. It caused the deaths of more than 2,400 people and destroyed tens of thousands of buildings."
DOCUMENT3 = "The government is also constantly revising the quake resistance levels required of buildings - identifying those that need upgrading. After 1999, they carried out a sweep of seismic retrofitting - that typically involved adding a framework of steel beams to a building's exterior, or adding reinforcements like extra pillars. That applied to infrastructure like bridges too.Prof Yih-Min Wu at the National Taiwan University, who has worked in the country's response and prevention departments for decades, says: Taiwan is hit so frequently by dangerous earthquakes [that] most of the poor quality buildings are already gone.And corrupt construction practices are prosecuted. Following the Tainan earthquake in the island's south-west in 2016, when a 17-storey high-rise collapsed killing dozens, five people involved in the building's construction were prosecuted and jailed.Of the 10 deaths reported so far this week, only one tragic loss of life occurred due to a collapse of a building - that 10-storey building in Hualien I mentioned earlier. The others were associated with landslides and rockfalls.And that points to another main reason for why Taiwan this time saw limited damage."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


**Creating the embedding database with ChromaDB**

API changes to Embeddings with model embedding-001
For the new embeddings model, embedding-001, there is a new task type parameter and the optional title (only valid with task_type=RETRIEVAL_DOCUMENT).

In [12]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [13]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [14]:
# Set up the DB
db = create_chroma_db(documents, "googlecarsdatabase")

In [15]:
pd.DataFrame(db.peek(3))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.02044009603559971, -0.05089940130710602, -0...",None,You can see it here in the city of Hualien. Ca...,None,None
1,1,"[0.006185354199260473, -0.046912286430597305, ...",None,"Just over a year ago, we saw earthquakes of ab...",None,None
2,2,"[0.006437219679355621, -0.0412362776696682, -0...",None,The government is also constantly revising the...,None,None


In [16]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [17]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db


In [19]:
# Set up the DB
db = create_chroma_db(documents, "BBCTaiwanNEWS")

**Confirm that the data was inserted by looking at the database:**

In [20]:
pd.DataFrame(db.peek(3))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.02044009603559971, -0.05089940130710602, -0...",None,You can see it here in the city of Hualien. Ca...,None,None
1,1,"[0.006185354199260473, -0.046912286430597305, ...",None,"Just over a year ago, we saw earthquakes of ab...",None,None
2,2,"[0.006437219679355621, -0.0412362776696682, -0...",None,The government is also constantly revising the...,None,None


**Getting the relevant document**
db is a Chroma collection object. You can call query on it to perform a nearest neighbors search to find similar embeddings or documents.

In [21]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

In [23]:
# Perform embedding search
passage = get_relevant_passage("earthquakes", db)
Markdown(passage)

Just over a year ago, we saw earthquakes of about the same magnitudes striking Turkey and Syria, causing the deaths of more than 50,000 people. These countries, of course, had far fewer resources. But when a much smaller 6.7 magnitude quake hit the city of Christchurch in New Zealand in 2011, almost the entire city centre was flattened.Taiwan is also often on the fault lines, but it has made significant progress in coping with shakes. Many say the wake-up call was Taiwan's 1999 Chi Chi earthquake - the worst in its history. It caused the deaths of more than 2,400 people and destroyed tens of thousands of buildings.

In [24]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

Pass a query to the prompt:

In [25]:
query = "Why did most of the Taiwan's building didn't get damage during this earthquakes"
prompt = make_prompt(query, passage)
Markdown(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'Why did most of the Taiwan's building didn't get damage during this earthquakes'
  PASSAGE: 'Just over a year ago, we saw earthquakes of about the same magnitudes striking Turkey and Syria, causing the deaths of more than 50,000 people. These countries, of course, had far fewer resources. But when a much smaller 6.7 magnitude quake hit the city of Christchurch in New Zealand in 2011, almost the entire city centre was flattened.Taiwan is also often on the fault lines, but it has made significant progress in coping with shakes. Many say the wake-up call was Taiwans 1999 Chi Chi earthquake - the worst in its history. It caused the deaths of more than 2,400 people and destroyed tens of thousands of buildings.'

    ANSWER:
  

**ANSWER:**
Now use the generate_content method to to generate a response from the model.

In [26]:
model = genai.GenerativeModel('gemini-pro')
answer = model.generate_content(prompt)
Markdown(answer.text)

The earthquake in Taiwan did not damage most buildings because Taiwan has made significant progress in coping with earthquakes since the devastating 1999 Chi Chi earthquake. This earthquake was a wake-up call for Taiwan, and the country has since implemented strict building codes and invested in earthquake-resistant construction techniques like reinforced concrete as well as early warning systems. These measures have helped to protect lives and property during subsequent earthquakes, including the recent 6.8-magnitude quake.